In [1]:
%load_ext autoreload
%autoreload 2

from deltas.pipeline import data, classifier, evaluation
from deltas.model import base, downsample
import deltas.plotting.plots as plots
from tqdm import tqdm

import numpy as np
import pandas as pd
# np.random.seed(10)


In [2]:

N1 = 1000
N2 = 10
m = 1
v = 1
costs = (1, 1)  # change for (1, 10) to increase results
# Gaussian (not always seperable)
dfs = []
for i in tqdm(range(10)):
    data_clf = data.get_data(
        m1=[-m, -m],
        m2=[m, m],
        cov1=[[v, 0], [0, v]],
        cov2=[[v, 0], [0, v]],
        N1=N1,
        N2=N2,
        scale=False,
        test_nums=[10000, 10000],
    )

    model = 'SVM-linear'
    model = 'SVM-rbf'
    # model = 'Linear'
    model = 'MLP'

    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                         alpha=10,
                                                         _print=False,
                                                         _plot=False,
                                                         method='supports-prop-update_mean',
                                                         max_trials=10000,
                                                         parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

 30%|███       | 3/10 [00:32<01:19, 11.39s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 40%|████      | 4/10 [00:44<01:10, 11.67s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 60%|██████    | 6/10 [01:08<00:47, 11.90s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 70%|███████   | 7/10 [01:20<00:35, 11.76s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/s

In [3]:
print(len(dfs))

10


In [4]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
print(mean_df)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])

                  Accuracy    G-Mean   ROC-AUC        F1
Baseline          0.573455  0.307330  0.959843  0.232669
SMOTE             0.842305  0.833383  0.942745  0.820422
Balanced Weights  0.805030  0.783350  0.926490  0.758625
BMR               0.873360  0.869049  0.959843  0.863686
Threshold         0.874100  0.870004  0.959843  0.864841
Our Method        0.873425  0.868862  0.959843  0.863071
\begin{tabular}{@{}lrrrr@{}}
\toprule
Methods & Accuracy & G-Mean & ROC-AUC & F1 \\
\midrule
Baseline & $.573 \pm .07$ & $.307 \pm .24$ & $\textbf{.959} \pm .00$ & $.232 \pm .21$ \\
SMOTE \cite{Chawla_2002_JAIR} & $.842 \pm .04$ & $.833 \pm .04$ & $.942 \pm .03$ & $.820 \pm .05$ \\
BW & $.805 \pm .06$ & $.783 \pm .08$ & $.926 \pm .06$ & $.758 \pm .10$ \\
BMR \cite{Bahnsen_2014_SIAM} & $.873 \pm .03$ & $.869 \pm .03$ & $.959 \pm .00$ & $.863 \pm .04$ \\
Thresh \cite{Sheng_2006_AAAI} & $\textbf{.874} \pm .02$ & $\textbf{.870} \pm .03$ & $.959 \pm .00$ & $\textbf{.864} \pm .04$ \\
Our Method & $.8

In [5]:
df.loc['BMR'].mean().to_list()
df.columns.to_list()

KeyError: 'BMR'